In [1]:
class Account:
    def __init__(self, account_number, balance):
        self.account_number = account_number
        self.balance = balance
    
    def print_info(self):
        print(f" Accountnumber: {self.account_number}, Balance: {self.balance}")
        
    def change_balance(self, new_balance):
        self.balance += new_balance #Den tar både plus och minus
        return self.balance
       
    def get_balance(self):
        return print(self.balance)
    
    def __str__(self):
        return f"Accountnumber: {self.account_number}, Balance: {self.balance}"
    
    def __repr__(self):
        return f"Accountnumber: {self.account_number}, Balance: {self.balance}"
    
    def make_dictionary_account(self):
        dictionary = {}
        dictionary["account_number"] = self.account_number
        dictionary["balance"] = self.balance
        return dictionary

In [2]:
class Customer:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.account_list = []
        self.account_number = 1
        
    def print_info(self):
        print(self.username, self.account_list)
        
    def check_password(self, password):
        if self.password == password:
            print("Right password.")
            return True
        print("Incorrect password.")
        return False
        
    def add_account(self, balance): #ska lägga till accounts till min privata lista
        self.account_number += 1
        self.account_list.append(Account(self.account_number, balance))
        print("Account added.")
        
    def __str__(self):
        return f"{self.username}, {self.password}"
    
    def __repr__(self):
        return f"username: {self.username}, password: {self.password}"   
    
    def make_dictionary_customer(self):
        dictionary = {}
        dictionary["username"] = self.username
        dictionary["password"] = self.password
        dictionary["account_list"] = []
        for account in self.account_list:
            dictionary["account_list"].append(account.make_dictionary_account())
        return dictionary

In [3]:
import json

class Bank:
    def __init__(self, bank_name):
        self.bank_name = bank_name
        self.customer_list = []
        self.current_customer = None
        self.bank_dictionary_list = []
        
    def online(self): #en variabel som ska innehålla inloggad kund, vissa funk ska endast fungera ifall kund är inloggad
        if self.current_customer != None:
            return True
        return False
    
    def add_account(self, balance = 0): #ska hitta rätt kund och sedan anv. add_acc från den customern
        if self.online():
            return self.current_customer.add_account(balance)
        else:
            return print("User not online.")
        
    def get_customers(self): #lista alla kunder
        for customer in self.customer_list:
            print(customer)
        
    def add_customer(self, username, password):
        if self.get_customer(username) == False:
            self.customer_list.append(Customer(username, password))
        else:
            return print("Username already exist.")
    
    def get_customer(self, sought_username):#hämta en kund via användarnamn
        for customer in self.customer_list:
            if customer.username == sought_username:
                return customer
        return False
                
    def change_customer_password(self, username, new_password): #loopa igenom användare, när dem hittat den vi letar efter sätter vi password == new_password
        if self.online():
            for customer in self.customer_list:
                if customer.username == username:
                    customer.password = new_password
                    print("Password successfully changed.")
                    return customer.password
                else: 
                    return print("Could not change password.")
        else:
            return print("User not logged in.")
    
    def remove_customer(self, username):
        for customer in self.customer_list:
            if customer.username == username:
                self.customer_list.remove(customer)
                print("Customer removed.")
    
    def login(self, username, password):
        for user in self.customer_list:        
            if username == user.username and password == user.password:
                self.current_customer = user
                print("User logged in.")
         
    def logout(self): #ta bort nuvarande anv från inloggade variabeln
        self.current_customer = None
        print("User logged out.")
    
    def get_accounts(self): #hämta alla konton som tillhör anv. som just nu är inloggad
        if self.online():
            return self.get_customer(self.current_customer.username).account_list
        else: 
            return print("User not online.")
    
    def remove_account(self, account_number): #ta bort ett konto för den kund som är inloggad
        if self.online():
            for account in self.current_customer.account_list:
                if account.account_number == account_number:
                    self.current_customer.account_list.remove(account)
                    return print("Success!")
                return print("Not a valid number.")
        else:
            return print("User not online.")
               
    
    def get_account(self, account_number):
        if self.online():
            for account in self.current_customer.account_list:
                if account.account_number == account_number:
                    return account
                return False
        else:
            return print("User not online.")
        
        
    def deposit(self, account_number, amount): #loopa igenom customer accounts
        if amount < 0:
            print("Can't be a negative number.")
            return
        if self.online() == True:
            for account in self.current_customer.account_list:
                if account.account_number == account_number:
                    account.change_balance(amount)
                    return print("Deposit succeeded.")
                return print(f"Account {account_number} could not be found.") 
        else:
            return print("User not online.")
    
    def withdraw(self, account_number, amount):
        if amount < 0:
            return print("Can't be a negative number.")
            if self.online():
                for account in self.current_customer.account_list:
                    if account.account_number == account_number:
                        if account.balance < amount:
                            return print("Insufficent funds.")
                    account.change_balance(- amount)
                    return print("Withdraw succeeded.")
            print(f"Account {account_number} could not be found.")
        else:
            return print("User not online.")
        
    def save_customer(self):
        file = open("bank.json", "w")
        for customer in self.customer_list:
            self.bank_dictionary_list.append(customer.make_dictionary_customer())
        json_string = json.dumps(self.bank_dictionary_list, indent = 2)
        file.write(json_string)
        file.close()
        self.customer_list.clear()
        self.bank_dictionary_list.clear()
                  
                  
    def load_customer(self):
        self.customer_list.clear()
        with open("bank.json", "r") as file:
            dictionary = json.load(file)
        for customer_dictionary in dictionary:
            customer = Customer(customer_dictionary["username"], customer_dictionary["password"])
            self.customer_list.append(customer)
            for account_dictionary in customer_dictionary["account_list"]:
                account = Account(account_dictionary["account_number"], account_dictionary["balance"])
                customer.account_list.append(account)
        for customer in self.customer_list:
            print(f"Customer: {customer.username}, Password: {customer.password}")
            for account in customer.account_list:
                print(f"Account: {account.account_number}, Balance: {account.balance}")
        file.close()


In [4]:
bank = Bank("Banken")

In [5]:
bank.add_customer("Julia", "lösen")
bank.add_customer("Malin", "lösen")
bank.add_customer("Joakim", "lösen")
bank.add_customer("Andreas", "lösen")

In [6]:
bank.get_customers()

Julia, lösen
Malin, lösen
Joakim, lösen
Andreas, lösen


In [7]:
bank.remove_customer("Andreas")

Customer removed.


In [8]:
bank.get_customer("Andreas")

False

In [9]:
bank.get_customers()

Julia, lösen
Malin, lösen
Joakim, lösen


In [10]:
bank.login("Julia", "lösen")

User logged in.


In [11]:
customer = Customer("Julia", "lösen")

In [12]:
customer.add_account(1)

Account added.


In [13]:
customer.add_account(114)

Account added.


In [14]:
bank.add_account(101)

Account added.


In [15]:
bank.get_accounts()

[Accountnumber: 2, Balance: 101]

In [16]:
bank.remove_account(1)

Not a valid number.


In [17]:
bank.remove_account(2)

Success!


In [18]:
bank.change_customer_password("Julia", "nyttlösen")

Password successfully changed.


'nyttlösen'

In [19]:
bank.get_accounts()

[]

In [20]:
bank.add_account(10)

Account added.


In [21]:
bank.add_account(10)

Account added.


In [22]:
bank.get_accounts()

[Accountnumber: 3, Balance: 10, Accountnumber: 4, Balance: 10]

In [23]:
bank.deposit(3, 10)

Deposit succeeded.


In [24]:
bank.deposit(3, 5)

Deposit succeeded.


In [25]:
bank.get_accounts()

[Accountnumber: 3, Balance: 25, Accountnumber: 4, Balance: 10]

In [26]:
bank.withdraw(3, 10)

User not online.


In [27]:
bank.get_accounts()

[Accountnumber: 3, Balance: 25, Accountnumber: 4, Balance: 10]

In [28]:
bank.withdraw(3, 100)

User not online.


In [29]:
customer.print_info()

Julia [Accountnumber: 2, Balance: 1, Accountnumber: 3, Balance: 114]


In [30]:
bank.logout()

User logged out.


In [31]:
bank.get_account(2)

User not online.


In [32]:
bank.get_accounts()

User not online.


In [33]:
bank.remove_account(2)

User not online.


In [34]:
bank.change_customer_password("Julia", "hej")

User not logged in.


In [35]:
bank.add_account("Julia")

User not online.


In [36]:
bank.login("Julia", "nyttlösen")

User logged in.


In [37]:
bank.login("Julia", "nyttlösen")

User logged in.


In [38]:
customer.print_info()

Julia [Accountnumber: 2, Balance: 1, Accountnumber: 3, Balance: 114]


In [39]:
bank.deposit(2, -100)

Can't be a negative number.


In [40]:
bank.withdraw(2, -100)

Can't be a negative number.


In [41]:
customer.print_info()

Julia [Accountnumber: 2, Balance: 1, Accountnumber: 3, Balance: 114]


In [42]:
account = Account(1, 10)

In [43]:
account.print_info()

 Accountnumber: 1, Balance: 10


In [44]:
account.print_info()

 Accountnumber: 1, Balance: 10


In [45]:
bank.get_accounts()

[Accountnumber: 3, Balance: 25, Accountnumber: 4, Balance: 10]

In [46]:
bank.get_customers()

Julia, nyttlösen
Malin, lösen
Joakim, lösen


In [47]:
bank.add_customer("Julia", "lösen")

Username already exist.


In [48]:
bank.get_customer("Julia")

username: Julia, password: nyttlösen

In [49]:
bank.get_customers()

Julia, nyttlösen
Malin, lösen
Joakim, lösen


In [50]:
bank.add_customer("hehe", "lösen")

In [51]:
bank.login("Julia", "lösen")

In [52]:
bank.logout()

User logged out.


In [53]:
bank.login("Malin", "lösen")

User logged in.


In [54]:
bank.login("Julia", "lösen")

In [55]:
bank.logout()

User logged out.


In [56]:
bank.save_customer()

In [57]:
bank.load_customer()

Customer: Julia, Password: nyttlösen
Account: 3, Balance: 25
Account: 4, Balance: 10
Customer: Malin, Password: lösen
Customer: Joakim, Password: lösen
Customer: hehe, Password: lösen
